In [1]:
import cv2
import numpy as np

lo=np.array([95, 100, 50])
hi=np.array([105, 255, 255])
color_infos=(0, 255, 255)
cap=cv2.VideoCapture(0) # Ouverture de la camera

while True: # Boucle infinie pour afficher les images de la camera
    ret, frame=cap.read() # Lecture de l'image de la camera et stockage dans frame (ret est un booleen qui indique si la lecture s'est bien passee)
    image=cv2.cvtColor(frame, cv2.COLOR_BGR2HSV) # Conversion de l'image de la camera en HSV (Hue Saturation Value) pour faciliter la detection de couleur
    mask=cv2.inRange(image, lo, hi) # Creation d'un masque pour detecter les couleurs dans l'intervalle lo-hi image binaire 
    image=cv2.blur(image, (7, 7)) # Floutage de l'image pour reduire le bruit et faciliter la detection
    mask=cv2.erode(mask, None, iterations=4) # Erosion du masque pour eliminer les petits elements parasites et faciliter la detection
    mask=cv2.dilate(mask, None, iterations=4) # Dilatation du masque pour rendre les objets plus visibles 
    image2=cv2.bitwise_and(frame, frame, mask=mask) # et bit à bit sur 2 images en utilisant le masque pour ne garder que les objets de la couleur voulue
    elements=cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[-2] # RETR_EXTERNAL pour ne garder que les contours exterieurs, CHAIN_APPROX_SIMPLE pour simplifier les contours 
    if len(elements) > 0: # Si des objets sont detectes donc > 0 elements
        c=max(elements, key=cv2.contourArea) # On prend le plus grand element (objet) par rapport a sa surface
        ((x, y), rayon)=cv2.minEnclosingCircle(c) # On cree un cercle autour de l'objet
        if rayon>30: # Si le rayon du cercle est superieur a 30 pixels
            cv2.circle(image2, (int(x), int(y)), int(rayon), color_infos, 2) # On dessine le cercle autour de l'objet
            cv2.circle(frame, (int(x), int(y)), 5, color_infos, 10) # On dessine un cercle au centre de l'objet
            cv2.line(frame, (int(x), int(y)), (int(x)+150, int(y)), color_infos, 2) # On dessine une ligne horizontale
            cv2.putText(frame, "Objet !!!", (int(x)+10, int(y) -10), cv2.FONT_HERSHEY_DUPLEX, 1, color_infos, 1, cv2.LINE_AA) # On ecrit "Objet !!!" a cote de l'objet
    cv2.imshow('Camera', frame)
    cv2.imshow('image2', image2)
    cv2.imshow('Mask', mask)
    if cv2.waitKey(1)&0xFF==ord('q'):
        break
cap.release()
cv2.destroyAllWindows()